# Data Analysis of Eye Tracking Data in Chest-Xrays



## Set Up

### Install requirements

In [1]:
# install required dependencies
#%pip install -r requirements.txt

### SETUP GPU

In [2]:
# setting up GPU
import tensorflow as tf
from util.test_gpu import cpu
from util.test_gpu import gpu
import timeit

print("Comparing CPU vs GPU executions...")
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('\n\nThis error most likely means that this notebook is not '
            'configured to use a GPU.  Change this in Notebook Settings via the '
            'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise SystemError('GPU device not found')
    
# We run each op once to warm up
#cpu()
#gpu()

# Run the op several times.
#print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
#    '(batch x height x width x channel). Sum of ten runs.')
#print('CPU (s):')

#cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
#print(cpu_time)
#print('GPU (s):')
#gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
#print(gpu_time)
#print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))


Comparing CPU vs GPU executions...
Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
0.7975776999999997
GPU (s):
0.03844570000000047
GPU speedup over CPU: 20x


### Importing Libraries

In [1]:
# general libs
import numpy as np
import pandas as pd
import pingouin as pt
import warnings
import math
import pickle
import shutil
import os

# image processing
from PIL import Image, ImageDraw, ImageFont
import cv2
import pydicom

# visualisation
import IPython
from IPython.core.display import display, HTML

# matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import pyplot, image
from matplotlib.patches import Ellipse

#seaborn
import seaborn as sns

# IPython/Jupyter Notebook progressbar decorator for iterators.
from tqdm.notebook import tqdm

# import my own functions
from  util.global_vars import GlobalVars
from mimicdata import *
from dataprocessing.patient import Patient

# no display limit in dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore') # ignore warnings

# will make your plot outputs appear and be stored within the notebook.
%matplotlib inline

sns.set() # sets seaborn as the main visualisationlib
sns.set_context('notebook') # Larger scale for plots in notebooks

# Plotting pretty figures and avoid blurry images
%config InlineBackend.figure_format = 'retina'

# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

plt.rcParams.update({'figure.max_open_warning': 1})

C:\Users\cmore\AppData\Local\Temp\ipykernel_28616\755396345.py:18: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Initializing Global Vars

In [2]:
# initialize vars
gv = GlobalVars( False )


In [3]:
process = MIMICDataProcessor()


MIMICDataProcessor


In [4]:
process.processEYE_GAZE( gv )

Loading EYE GAZE original dataset...


In [5]:
PATIENTS_DIC = gv.getPATIENTS_DIC()
PATIENTS_DIC

{'15628804_IMG1': <dataprocessing.patient.Patient at 0x242f7df36a0>,
 '19462352_IMG1': <dataprocessing.patient.Patient at 0x242f6b637f0>,
 '11255143_IMG1': <dataprocessing.patient.Patient at 0x242a35d40a0>}

In [8]:
PATIENTS_DIC['19462352_IMG1'].getPatient_data()

{'STUDY_ID': 's51900589',
 'IMAGE_ID': 's78711a04-264d5305-d5feec9b-ebef1cec-fdc6db9c',
 'PATIENT_ID': '19462352',
 'PATIENT_KEY': '19462352_IMG1',
 'METADATA': 'D:GoogleDrive\\Datasets\\XAMI-MIMIC\\XAMI-MIMIC\\patient_19462352\\EyeGaze\\master_sheet.csv',
 'CORE_PATH': 'D:GoogleDrive\\Datasets\\XAMI-MIMIC\\XAMI-MIMIC\\patient_19462352\\Core\\',
 'ED': 'D:GoogleDrive\\Datasets\\XAMI-MIMIC\\XAMI-MIMIC\\patient_19462352\\ED\\',
 'EYE_GAZE_PATH': 'D:GoogleDrive\\Datasets\\XAMI-MIMIC\\XAMI-MIMIC\\patient_19462352\\EyeGaze\\',
 'DIAGNOSIS': {'COMMENTS': '___F with hypertension, tachycardia',
  'KEY_CONDITIONS_NLP': ['pleural_effusion_or_thickening',
   'pulmonary_edema__hazy_opacity',
   'vascular_redistribution'],
  'KEY_CONDITIONS_CHEXPERT_PRED': ['edema', 'pleural_effusion', 'pneumonia']}}